# Project: Data Analysis on employees

- Task 1: Generate DataFrame from CSV data.
- Task 2: Define a schema for the data.
- Task 3: Display schema of DataFrame.
- Task 4: Create a temporary view.
- Task 5: Execute an SQL query.
- Task 6: Calculate Average Salary by Department.
- Task 7: Filter and Display IT Department Employees.
- Task 8: Add 10% Bonus to Salaries.
- Task 9: Find Maximum Salary by Age.
- Task 10: Self-Join on Employee Data.
- Task 11: Calculate Average Employee Age.
- Task 12: Calculate Total Salary by Department.
- Task 13: Sort Data by Age and Salary.
- Task 14: Count Employees in Each Department.
- Task 15: Filter Employees with the letter o in the Name.


In [1]:
# Installing required packages 
!pip install pyspark  findspark wget

,Requirement already satisfied: findspark in /home/jupyterlab/conda/envs/python/lib/python3.7/site-packages (2.0.1)
,Requirement already satisfied: wget in /home/jupyterlab/conda/envs/python/lib/python3.7/site-packages (3.2)
,Requirement already satisfied: py4j==0.10.9 in /home/jupyterlab/conda/envs/python/lib/python3.7/site-packages (from pyspark) (0.10.9)


In [2]:
import findspark

findspark.init()

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [4]:
# Creating a SparkContext object  

sc = SparkContext.getOrCreate()

# Creating a SparkSession  

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

25/08/18 23:34:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
,Setting default log level to "WARN".
,To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Download the CSV data first into a local `employees.csv` file
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/data/employees.csv")

'employees (1).csv'

### Tasks


#### Task 1: Generate a Spark DataFrame from the CSV data

In [6]:
# Read data from the "emp" CSV file and import it into a DataFrame variable named "employees_df"  
employees_df1 = spark.read.csv("employees.csv", header=True, inferSchema=True)


#### Task 2: Define a schema for the data

In [7]:
# Schema for the input data and read the file using the user-defined Schema
schema = StructType([
    StructField("Emp_No", IntegerType(), True),
    StructField("Emp_name", StringType(), True),
    StructField("Salary", IntegerType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Department", StringType(), True)
])
employees_df2 = spark.read.csv("employees.csv", schema=schema, header=True)



#### Task 3: Display schema of DataFrame

In [8]:
# Display all columns of the DataFrame, along with their respective data types
employees_df2.printSchema()

root
, |-- Emp_No: integer (nullable = true)
, |-- Emp_name: string (nullable = true)
, |-- Salary: integer (nullable = true)
, |-- Age: integer (nullable = true)
, |-- Department: string (nullable = true)
,


#### Task 4: Create a temporary view

In [9]:
# Create a temporary view named "employees" for the DataFrame
spark.sql("DROP VIEW IF EXISTS employees")

employees_df2.createTempView('employees')

#### Task 5: Execute an SQL query

In [10]:
# SQL query to fetch solely the records from the View where the age exceeds 30
spark.sql('SELECT * FROM employees WHERE Age > 30').show()

+------+-----------+------+---+----------+
,|Emp_No|   Emp_name|Salary|Age|Department|
,+------+-----------+------+---+----------+
,|   199|    Douglas|  2600| 34|     Sales|
,|   200|   Jennifer|  4400| 36| Marketing|
,|   201|    Michael| 13000| 32|        IT|
,|   202|        Pat|  6000| 39|        HR|
,|   203|      Susan|  6500| 36| Marketing|
,|   205|    Shelley| 12008| 33|   Finance|
,|   206|    William|  8300| 37|        IT|
,|   100|     Steven| 24000| 39|        IT|
,|   102|        Lex| 17000| 37| Marketing|
,|   103|  Alexander|  9000| 39| Marketing|
,|   104|      Bruce|  6000| 38|        IT|
,|   105|      David|  4800| 39|        IT|
,|   106|      Valli|  4800| 38|     Sales|
,|   107|      Diana|  4200| 35|     Sales|
,|   109|     Daniel|  9000| 35|        HR|
,|   110|       John|  8200| 31| Marketing|
,|   111|     Ismael|  7700| 32|        IT|
,|   112|Jose Manuel|  7800| 34|        HR|
,|   113|       Luis|  6900| 34|     Sales|
,|   116|     Shelli|  2900| 37| 

#### Task 6: Calculate Average Salary by Department

In [11]:
# SQL query to calculate the average salary of employees grouped by department
spark.sql('SELECT Department, ROUND(AVG(Salary),2) AS avg_Salary FROM employees GROUP BY Department').show()

+----------+----------+
,|Department|avg_Salary|
,+----------+----------+
,|     Sales|   5492.92|
,|        HR|    5837.5|
,|   Finance|    5730.8|
,| Marketing|   6633.33|
,|        IT|    7400.0|
,+----------+----------+
,


#### Task 7: Filter and Display IT Department Employees

In [12]:
# Apply a filter to select records where the department is 'IT'
employees_df2.filter("Department = 'IT'").show()

+------+--------+------+---+----------+
,|Emp_No|Emp_name|Salary|Age|Department|
,+------+--------+------+---+----------+
,|   198|  Donald|  2600| 29|        IT|
,|   201| Michael| 13000| 32|        IT|
,|   206| William|  8300| 37|        IT|
,|   100|  Steven| 24000| 39|        IT|
,|   104|   Bruce|  6000| 38|        IT|
,|   105|   David|  4800| 39|        IT|
,|   111|  Ismael|  7700| 32|        IT|
,|   129|   Laura|  3300| 38|        IT|
,|   132|      TJ|  2100| 34|        IT|
,|   136|   Hazel|  2200| 29|        IT|
,+------+--------+------+---+----------+
,


#### Task 8: Add 10% Bonus to Salaries

In [13]:
from pyspark.sql.functions import col

# Add a new column "SalaryAfterBonus" with 10% bonus added to the original salary
employees_df2 = employees_df2.withColumn("SalaryAfterBonus", col("Salary") * 1.1)
employees_df2.show(5)

+------+--------+------+---+----------+------------------+
,|Emp_No|Emp_name|Salary|Age|Department|  SalaryAfterBonus|
,+------+--------+------+---+----------+------------------+
,|   198|  Donald|  2600| 29|        IT|2860.0000000000005|
,|   199| Douglas|  2600| 34|     Sales|2860.0000000000005|
,|   200|Jennifer|  4400| 36| Marketing|            4840.0|
,|   201| Michael| 13000| 32|        IT|14300.000000000002|
,|   202|     Pat|  6000| 39|        HR| 6600.000000000001|
,+------+--------+------+---+----------+------------------+
,only showing top 5 rows
,


#### Task 9: Find Maximum Salary by Age

In [14]:
from pyspark.sql.functions import max

# Group data by age and calculate the maximum salary for each age group
employees_df2.groupby("Age").agg(max("Salary").alias("Max_Salary")).show()

+---+----------+
,|Age|Max_Salary|
,+---+----------+
,| 31|      8200|
,| 34|      7800|
,| 28|     12008|
,| 27|     17000|
,| 26|      3600|
,| 37|     17000|
,| 35|      9000|
,| 39|     24000|
,| 38|      6000|
,| 29|     10000|
,| 32|     13000|
,| 33|     12008|
,| 30|      8000|
,| 36|      7900|
,+---+----------+
,


#### Task 10: Self-Join on Employee Data

In [15]:
# Join the DataFrame with itself based on the "Emp_No" column
employees_df2_joined = employees_df2.join(employees_df2, "Emp_No")
employees_df2_joined.show()


+------+---------+------+---+----------+------------------+---------+------+---+----------+------------------+
,|Emp_No| Emp_name|Salary|Age|Department|  SalaryAfterBonus| Emp_name|Salary|Age|Department|  SalaryAfterBonus|
,+------+---------+------+---+----------+------------------+---------+------+---+----------+------------------+
,|   198|   Donald|  2600| 29|        IT|2860.0000000000005|   Donald|  2600| 29|        IT|2860.0000000000005|
,|   199|  Douglas|  2600| 34|     Sales|2860.0000000000005|  Douglas|  2600| 34|     Sales|2860.0000000000005|
,|   200| Jennifer|  4400| 36| Marketing|            4840.0| Jennifer|  4400| 36| Marketing|            4840.0|
,|   201|  Michael| 13000| 32|        IT|14300.000000000002|  Michael| 13000| 32|        IT|14300.000000000002|
,|   202|      Pat|  6000| 39|        HR| 6600.000000000001|      Pat|  6000| 39|        HR| 6600.000000000001|
,|   203|    Susan|  6500| 36| Marketing| 7150.000000000001|    Susan|  6500| 36| Marketing| 7150.0000000

#### Task 11: Calculate Average Employee Age

In [16]:
# Calculate the average age of employees
from pyspark.sql.functions import avg 
employees_df2.agg(avg("Age")).show()


+--------+
,|avg(Age)|
,+--------+
,|   33.56|
,+--------+
,


#### Task 12: Calculate Total Salary by Department

In [17]:
# Calculate the total salary for each department. Hint - User GroupBy and Aggregate functions
from pyspark.sql.functions import sum 
employees_df2.groupby("Department").agg(sum("Salary").alias("Total_Salary")).show()
employees_df2.groupby("Department").agg({"Salary": "SUM"}).show()

+----------+------------+
,|Department|Total_Salary|
,+----------+------------+
,|     Sales|       71408|
,|        HR|       46700|
,|   Finance|       57308|
,| Marketing|       59700|
,|        IT|       74000|
,+----------+------------+
,


+----------+-----------+
,|Department|sum(Salary)|
,+----------+-----------+
,|     Sales|      71408|
,|        HR|      46700|
,|   Finance|      57308|
,| Marketing|      59700|
,|        IT|      74000|
,+----------+-----------+
,


#### Task 13: Sort Data by Age and Salary

In [18]:
# Sort the DataFrame by age in ascending order and then by salary in descending order
employees_df2.orderBy(col("Age").asc(), col("Salary").desc()).show()

+------+---------+------+---+----------+------------------+
,|Emp_No| Emp_name|Salary|Age|Department|  SalaryAfterBonus|
,+------+---------+------+---+----------+------------------+
,|   137|   Renske|  3600| 26| Marketing|3960.0000000000005|
,|   101|    Neena| 17000| 27|     Sales|           18700.0|
,|   114|      Den| 11000| 27|   Finance|12100.000000000002|
,|   108|    Nancy| 12008| 28|     Sales|13208.800000000001|
,|   130|    Mozhe|  2800| 28| Marketing|3080.0000000000005|
,|   126|    Irene|  2700| 28|        HR|2970.0000000000005|
,|   204|  Hermann| 10000| 29|   Finance|           11000.0|
,|   115|Alexander|  3100| 29|   Finance|3410.0000000000005|
,|   134|  Michael|  2900| 29|     Sales|3190.0000000000005|
,|   198|   Donald|  2600| 29|        IT|2860.0000000000005|
,|   140|   Joshua|  2500| 29|   Finance|            2750.0|
,|   136|    Hazel|  2200| 29|        IT|            2420.0|
,|   120|  Matthew|  8000| 30|        HR|            8800.0|
,|   110|     John|  8200

#### Task 14: Count Employees in Each Department

In [19]:
from pyspark.sql.functions import count

# Calculate the number of employees in each department
employees_df2.groupBy("Department").agg({"Emp_No": "COUNT"}).show()
employees_df2.groupBy("Department").agg(count("Emp_No")).show()

+----------+-------------+
,|Department|count(Emp_No)|
,+----------+-------------+
,|     Sales|           13|
,|        HR|            8|
,|   Finance|           10|
,| Marketing|            9|
,|        IT|           10|
,+----------+-------------+
,


+----------+-------------+
,|Department|count(Emp_No)|
,+----------+-------------+
,|     Sales|           13|
,|        HR|            8|
,|   Finance|           10|
,| Marketing|            9|
,|        IT|           10|
,+----------+-------------+
,


#### Task 15: Filter Employees with the letter o in the Name

In [20]:
# Apply a filter to select records where the employee's name contains the letter 'o'
spark.sql("SELECT * FROM employees WHERE Emp_Name LIKE '%o%'").show()

+------+-----------+------+---+----------+
,|Emp_No|   Emp_name|Salary|Age|Department|
,+------+-----------+------+---+----------+
,|   198|     Donald|  2600| 29|        IT|
,|   199|    Douglas|  2600| 34|     Sales|
,|   110|       John|  8200| 31| Marketing|
,|   112|Jose Manuel|  7800| 34|        HR|
,|   130|      Mozhe|  2800| 28| Marketing|
,|   133|      Jason|  3300| 38|     Sales|
,|   139|       John|  2700| 36|     Sales|
,|   140|     Joshua|  2500| 29|   Finance|
,+------+-----------+------+---+----------+
,


<!--## Change Log -->


<!--|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-09-01|0.1|Lavanya T S|Initial version|
|2023-09-11|0.2|Pornima More|QA pass with edits|-->
